# Homework #3 - Rossman Store Sales

Group Members:

Hoo De Lin (A0126576X)

Yap Han Chiang (A0125168E)

# Analyzing the data:

Before we try figuring out which model to fit in, we need to understand the data, and clean/modify the data to be able to feed into any model

The feature that I believe are quite important are:

- Competition
- Holidays
- Type of stores
- Type of day (Monday - Sunday)
- Having promotions
- Promo2 since

Interesting to note is that, 

In [156]:
import numpy as np
import pandas as pd
import math
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as pl
%matplotlib inline
import seaborn as sb 
import csv
from sklearn.ensemble import RandomForestRegressor

# import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, mean_squared_error

In [219]:
trainData = pd.read_csv('train_v2.csv', parse_dates=['Date'], dtype={'StateHoliday': 'str'})
storeData = pd.read_csv('store.csv')
testData = pd.read_csv('test_v2.csv', parse_dates=['Date'], dtype={'StateHoliday': 'str', 'StoreType': 'str', 'Assortment': 'str'})
trainData = pd.merge(storeData, trainData, on='Store')
testData = testData.reset_index().merge(storeData,on='Store')
index = testData['index']
testData.drop('index', axis=1, inplace=True)

In [186]:
trainData

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,5,2015-01-30,5577,616,1,1,0,0
1,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,4,2015-01-29,5171,573,1,1,0,0
2,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,3,2015-01-28,4310,537,1,1,0,0
3,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2,2015-01-27,4806,532,1,1,0,0
4,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,1,2015-01-26,4781,566,1,1,0,0
5,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,7,2015-01-25,0,0,0,0,0,0
6,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,6,2015-01-24,4840,600,1,0,0,0
7,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,5,2015-01-23,3586,489,1,0,0,0
8,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,4,2015-01-22,3492,493,1,0,0,0
9,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,3,2015-01-21,3299,460,1,0,0,0


In [220]:
# Cleaning the data
competition_distance_mean = trainData['CompetitionDistance'].mean()
customer_mean = trainData['Customers'].mean()
competition_distance_std = trainData['CompetitionDistance'].std()
customer_std = trainData['Customers'].std()

def normalizeDistance(row):
    return (row['CompetitionDistance'] - competition_distance_mean) / competition_distance_std
def normalizeCustomer(row):
    return (row['Customers'] - customer_mean) / customer_std
def hasHoliday(row):
    if str(row['StateHoliday']) != "0" or str(row['SchoolHoliday']) != "0":
        return 1
    else:
        return 0
def calculateCompetitionAge(row):
        if math.isnan(row['CompetitionOpenSinceMonth']):
            return 0
        else:
            f_date = datetime(int(row['CompetitionOpenSinceYear']), int(row['CompetitionOpenSinceMonth']), 1, 0, 0, 0)
            return math.floor(((row['Date'].to_pydatetime() - f_date).days) / 30)

month_num_name_dict = {
    "1": "Jan",
    "2": "Feb",
    "3": "Mar",
    "4": "Apr",
    "5": "May",
    "6": "Jun",
    "7": "Jul",
    "8": "Aug",
    "9": "Sept",
    "10": "Oct",
    "11": "Nov",
    "12": "Dec"
}

def invert(row):
    return 1 / row['CompetitionDistance']

def isHavingPromo2(row):
    if not isinstance(row['PromoInterval'], str):
        return 0
    elif month_num_name_dict[str(row['Date'].to_pydatetime().month)] in row['PromoInterval']:
        return 1
    else:
        return 0

# Possible thought: If we are not at the promo interval, should promo2Age have any effect?
def promo2Age(row):
    if row['Promo2'] == 0:
        return 0
    else:
#        https://stackoverflow.com/a/17087427
        date = str(int(row["Promo2SinceYear"])) + "-W" + str(int(row["Promo2SinceWeek"]))
        f_date = datetime.strptime(date + '-0', "%Y-W%W-%w")
        return math.floor((row['Date'].to_pydatetime() - f_date).days/ 30)
mapCharToInt = {'0': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4}
mapPromoIntervalToInt = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7,
                        'Aug': 8, 'Sept': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
def getDayOfMonth(row):
    return str(row['Date']).split('-')[2].split(' ')[0]

def getMonth(row):
    return row['Date'].month

def isTomorrowHoliday(row):
    current = row['isTomorrowHoliday']
    if (row['Date'].to_pydatetime() + timedelta(1) == row['previousRowDate']):
        return current
    else:
        return 0
def isYesterdayHoliday(row):
    current = row['isYesterdayHoliday']
    if (row['Date'].to_pydatetime() - timedelta(1) == row['nextRowDate']):
        return current
    else:
        return 0

In [196]:
trainData

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,HasHoliday
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,5,2015-01-30,5577,616,1,1,0,0,0
1,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,4,2015-01-29,5171,573,1,1,0,0,0
2,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,3,2015-01-28,4310,537,1,1,0,0,0
3,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2,2015-01-27,4806,532,1,1,0,0,0
4,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,1,2015-01-26,4781,566,1,1,0,0,0
5,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,7,2015-01-25,0,0,0,0,0,0,0
6,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,6,2015-01-24,4840,600,1,0,0,0,0
7,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,5,2015-01-23,3586,489,1,0,0,0,0
8,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,4,2015-01-22,3492,493,1,0,0,0,0
9,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,3,2015-01-21,3299,460,1,0,0,0,0


In [221]:
trainData['HasHoliday'] = trainData.apply(lambda row: hasHoliday(row), axis=1)
trainData['previousRowDate'] = trainData['Date'].shift(1)
trainData['nextRowDate'] = trainData['Date'].shift(-1)
trainData['isTomorrowHoliday'] = trainData['HasHoliday'].shift(1)
trainData['isYesterdayHoliday'] = trainData['HasHoliday'].shift(-1)
trainData['isTomorrowHoliday'] = trainData.apply(lambda row: isTomorrowHoliday(row), axis=1)
trainData['isYesterdayHoliday'] = trainData.apply(lambda row: isYesterdayHoliday(row), axis=1)
trainData.drop(['previousRowDate','nextRowDate'],inplace=True, axis=1)
trainData['CompetitionAge'] = trainData.apply(lambda row: calculateCompetitionAge(row), axis=1)
trainData['HavingPromo2'] = trainData.apply(lambda row: isHavingPromo2(row), axis=1)
trainData['Promo2Age'] = trainData.apply(lambda row: promo2Age(row), axis=1)
trainData['CompetitionDistance'] = trainData.apply(lambda row: invert(row), axis=1)
trainData['DayOfMonth'] = trainData.apply(lambda row: getDayOfMonth(row), axis=1)
trainData['Month'] = trainData.apply(lambda row: getMonth(row), axis=1)
trainData.replace(mapCharToInt, inplace=True)
trainData.fillna(0, inplace=True)
testData['HasHoliday'] = testData.apply(lambda row: hasHoliday(row), axis=1)
testData['previousRowDate'] = testData['Date'].shift(1)
testData['nextRowDate'] = testData['Date'].shift(-1)
testData['isTomorrowHoliday'] = testData['HasHoliday'].shift(1)
testData['isYesterdayHoliday'] = testData['HasHoliday'].shift(-1)
testData['isTomorrowHoliday'] = testData.apply(lambda row: isTomorrowHoliday(row), axis=1)
testData['isYesterdayHoliday'] = testData.apply(lambda row: isYesterdayHoliday(row), axis=1)
testData.drop(['previousRowDate','nextRowDate'],inplace=True, axis=1)
testData['CompetitionAge'] = testData.apply(lambda row: calculateCompetitionAge(row), axis=1)
testData['HavingPromo2'] = testData.apply(lambda row: isHavingPromo2(row), axis=1)
testData['Promo2Age'] = testData.apply(lambda row: promo2Age(row), axis=1)
testData['CompetitionDistance'] = testData.apply(lambda row: invert(row), axis=1)
testData['DayOfMonth'] = testData.apply(lambda row: getDayOfMonth(row), axis=1)
testData['Month'] = testData.apply(lambda row: getMonth(row), axis=1)
testData.replace(mapCharToInt, inplace=True)
testData.fillna(0, inplace=True)

In [218]:
trainData

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,...,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,HasHoliday,isTomorrowHoliday,isYesterdayHoliday
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,...,2015-01-30,5577,616,1,1,0,0,0,0.0,0.0
1,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,...,2015-01-29,5171,573,1,1,0,0,0,0.0,0.0
2,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,...,2015-01-28,4310,537,1,1,0,0,0,0.0,0.0
3,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,...,2015-01-27,4806,532,1,1,0,0,0,0.0,0.0
4,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,...,2015-01-26,4781,566,1,1,0,0,0,0.0,0.0
5,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,...,2015-01-25,0,0,0,0,0,0,0,0.0,0.0
6,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,...,2015-01-24,4840,600,1,0,0,0,0,0.0,0.0
7,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,...,2015-01-23,3586,489,1,0,0,0,0,0.0,0.0
8,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,...,2015-01-22,3492,493,1,0,0,0,0,0.0,0.0
9,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,...,2015-01-21,3299,460,1,0,0,0,0,0.0,0.0


In [ ]:
testData

In [222]:
# Dropping features
_trainData = trainData.drop(['CompetitionOpenSinceMonth','CompetitionOpenSinceYear','StateHoliday','SchoolHoliday','Promo2SinceWeek', 'Promo2SinceYear','PromoInterval', 'Promo2', 'Open'], axis=1)
_testData = testData.drop(['CompetitionOpenSinceMonth','CompetitionOpenSinceYear','StateHoliday','SchoolHoliday','Promo2SinceWeek', 'Promo2SinceYear','PromoInterval', 'Promo2', 'Open'], axis=1)
y = _trainData[_trainData.Customers != 0].Sales.values
__trainData = _trainData[_trainData.Customers != 0]
_trainData = _trainData[_trainData.Customers != 0].drop(['Date','Sales', 'Store'], axis=1)
_testData = _testData.drop(['Date','Store'], axis=1)

In [223]:
_trainData.head()

,StoreType,Assortment,CompetitionDistance,DayOfWeek,Customers,Promo,HasHoliday,isTomorrowHoliday,isYesterdayHoliday,CompetitionAge,HavingPromo2,Promo2Age,DayOfMonth,Month
0,3,1,0.000787,5,616,1,0,0.0,0.0,78,0,0,30,1
1,3,1,0.000787,4,573,1,0,0.0,0.0,78,0,0,29,1
2,3,1,0.000787,3,537,1,0,0.0,0.0,78,0,0,28,1
3,3,1,0.000787,2,532,1,0,0.0,0.0,77,0,0,27,1
4,3,1,0.000787,1,566,1,0,0.0,0.0,77,0,0,26,1


In [224]:
_testData.head()

,DayOfWeek,Customers,Promo,StoreType,Assortment,CompetitionDistance,HasHoliday,isTomorrowHoliday,isYesterdayHoliday,CompetitionAge,HavingPromo2,Promo2Age,DayOfMonth,Month
0,5,555,1,3,1,0.000787,1,0.0,1.0,84,0,0,31,7
1,4,546,1,3,1,0.000787,1,1.0,1.0,84,0,0,30,7
2,3,523,1,3,1,0.000787,1,1.0,1.0,84,0,0,29,7
3,2,560,1,3,1,0.000787,1,1.0,1.0,84,0,0,28,7
4,1,612,1,3,1,0.000787,1,1.0,0.0,84,0,0,27,7


In [231]:
X = _trainData.values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)
rfr = RandomForestRegressor(n_estimators=3)
clf = rfr.fit(X_train, y_train)
print(clf.score(X_val, y_val))
rfr.fit(X,y)
print(rfr.feature_importances_)

0.912288923005
[  5.38710305e-02   1.95901317e-02   6.20320763e-02   8.61545975e-03
   7.70779077e-01   1.99023990e-02   7.40763399e-04   1.74425357e-03
   1.10035244e-03   2.05989830e-02   2.06844455e-03   1.92656431e-02
   9.57822773e-03   1.01131576e-02]


In [232]:
results = rfr.predict(_testData.as_matrix())
store_closed_index = np.where(_testData.Customers == 0)

print(store_closed_index)

for val in store_closed_index:
    results[val] = 0
with open('submission.csv', 'w') as csvfile:
    fieldnames = ['Id', 'Sales']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for idx, val in enumerate(results):
        writer.writerow({'Id': index[idx] + 1, 'Sales': val})
    csvfile.close()

(array([    5,    12,    19, ..., 33432, 33439, 33446]),)


How bout some xgboost?

In [228]:
X = _trainData.values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)
xgb = XGBRegressor()
clf = xgb.fit(X_train, y_train)
print(clf.score(X_val, y_val))
xgb.fit(X,y)
xgb = xgb.predict(_testData.as_matrix())
store_closed_index = np.where(_testData.Customers == 0)

print(store_closed_index)

for val in store_closed_index:
    results[val] = 0
with open('submission.csv', 'w') as csvfile:
    fieldnames = ['Id', 'Sales']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for idx, val in enumerate(results):
        writer.writerow({'Id': index[idx] + 1, 'Sales': val})
    csvfile.close()

0.903420446558
(array([    5,    12,    19, ..., 33432, 33439, 33446]),)


Previously, we have take account daily trend, but not monthly trends and sales trend